In [1]:
import copy
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Matching

In [2]:
def draw_registration_result(source, target, transformation= np.identity(4)):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)

    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      zoom=0.4459,
                                      front=[0.9288, -0.2951, -0.2242],
                                      lookat=[1.6784, 2.0612, 1.4451],
                                      up=[-0.3402, -0.9189, -0.1996])

def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

def prepare_dataset(source, target, voxel_size = 0.05):
    print(":: Load two point clouds and disturb initial pose.")

    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source_down, target_down, source_fpfh, target_fpfh

def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    print(":: RANSAC registration on downsampled point clouds.")
    print("   Since the downsampling voxel size is %.3f," % voxel_size)
    print("   we use a liberal distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True,
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        3, [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
                0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold)
        ], o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999))
    return result

def refine_registration(source, target, result_ransac, voxel_size):
    distance_threshold = voxel_size * 0.4
    print(":: Point-to-plane ICP registration is applied on original point")
    print("   clouds to refine the alignment. This time we use a strict")
    print("   distance threshold %.3f." % distance_threshold)
    radius_normal = voxel_size * 2
    source.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
    target.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
    result = o3d.pipelines.registration.registration_icp(
        source, target, distance_threshold, result_ransac.transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPlane())
    return result


### Change detection

In [3]:
#Import point clouds
source_pcd = o3d.io.read_point_cloud("TestData/test_before/pcd/map_go_5.pcd")
target_pcd = o3d.io.read_point_cloud("TestData/test_after/pcd/map_go_5.pcd")

#Prepare dataset & Apply transformations (RANSAC & ICP)
voxel_size=0.5

source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(source_pcd, target_pcd, voxel_size)
result_ransac = execute_global_registration(source_down, target_down,
                                            source_fpfh, target_fpfh,
                                            voxel_size)

result_icp = refine_registration(source_pcd, target_pcd, result_ransac,
                                 voxel_size)
#Store transormation matrix
T = result_icp.transformation
print(T)
np.save("TestData/test_after/pcd/T",T)

:: Load two point clouds and disturb initial pose.
:: Downsample with a voxel size 0.500.
:: Estimate normal with search radius 1.000.
:: Compute FPFH feature with search radius 2.500.
:: Downsample with a voxel size 0.500.
:: Estimate normal with search radius 1.000.
:: Compute FPFH feature with search radius 2.500.
:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 0.500,
   we use a liberal distance threshold 0.750.
:: Point-to-plane ICP registration is applied on original point
   clouds to refine the alignment. This time we use a strict
   distance threshold 0.200.
[[ 0.92043419  0.39089269  0.00194994  0.09573761]
 [-0.39089046  0.92037594  0.01061944  0.27213607]
 [ 0.00235638 -0.01053671  0.99994171 -0.00349512]
 [ 0.          0.          0.          1.        ]]


In [4]:
# target_pcd_t = copy.deepcopy(target_pcd)
source_pcd_t = copy.deepcopy(source_pcd)
source_pcd_t.transform(T)
o3d.io.write_point_cloud("TestData/test_before/pcd/source_pcd_t.pcd",source_pcd_t)
draw_registration_result(source_pcd_t, target_pcd)

In [5]:
# print("Testing kdtree in Open3D...")
# print("Load a point cloud and paint it gray.")

# source_pcd.paint_uniform_color([0.5, 0.5, 0.5])
# pcd_tree = o3d.geometry.KDTreeFlann(source_pcd)

# print("Paint the 1501st point red.")
# source_pcd.colors[1500] = [1, 0, 0]

# print("Find its 200 nearest neighbors, and paint them blue.")
# [k, idx, _] = pcd_tree.search_knn_vector_3d(source_pcd.points[1500], 200)
# np.asarray(source_pcd.colors)[idx[1:], :] = [0, 0, 1]

# print("Visualize the point cloud.")
# o3d.visualization.draw_geometries([source_pcd],
#                                   zoom=0.5599,
#                                   front=[-0.4958, 0.8229, 0.2773],
#                                   lookat=[2.1126, 1.0163, -1.8543],
#                                   up=[0.1007, -0.2626, 0.9596])

Testing kdtree in Open3D...
Load a point cloud and paint it gray.
Paint the 1501st point red.
Find its 200 nearest neighbors, and paint them blue.
Visualize the point cloud.
